# Evolutionary Strategies

In order to understand how we'll be evolving the FPGA to approximate any function, let's start of with a simple problem. We will try to evolve random bit strings to approximate the phrase "`Hello, World!`". We will encode "`Hello, World!`" into a stream of binary 7-bit ASCII strings. 

### Table of Contents
1. [Simple Genetic Algo](#sga)

In [100]:
# Libraries
import operator
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(seed=100)

### Helper functions

In [94]:
# Return a random binary list of length N
# drawn from uniform distribution
random_bits = lambda N: np.random.choice([0, 1], size=(N,), p=[0.5, 0.5])

# Converts text to bit string
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

# Converts bit string to normal text
def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

In [92]:
text_from_bits(text_to_bits('Hello, World!'))

'Hello, World!'

---
## Simple Genetic Algorithm<a id='sga'></a>


Get Target word's bit-string representation

In [93]:
target_word = 'Hello, World!'
target_word_bits = np.array([int(bit) for bit in text_to_bits(target_word)])

### 1. Create Population: Generate Random Individuals

In [98]:
pop_size = 100
population = [[bit for bit in random_bits(len(target_word_bits))] for _ in range(pop_size)]

Since our population is represented as a matrix, it might be possible to use matrix operations to compute the fitness of each individual (row of matrix).

### 2. Get Fitness of individiuals

Let's define a fitness function. Because we'll be evaluating how much a bit-string matches another bit-string, we'll use the cosine similarity as a measure of fitness:

$$
\begin{aligned}
cos(\theta) &= \frac{\vec{\mathbf{a}} \cdot \vec{\mathbf{b}}}{\vert\vert\vec{\mathbf{a}}\vert\vert\cdot\vert\vert\vec{\mathbf{b}}\vert\vert}, -1 \leq cos(\theta) \leq 1
\end{aligned}
$$

In [41]:
def fitness(individual, target_word_bits=target_word_bits):
    '''
    Function:
    ---------
    Determines the fitness of an individual based
    on cosine similarity to bit-string
    
    Parameters:
    -----------
    individual: (list) List of 0s and 1s
    target_word_bits: (list) list of 0s and 1s representing the target word 
    
    Returns:
    --------
    fitness score (cosine similarity) of the inidividual
    '''
    
    # Function to calculate cosine similarity
    # between 2 vectors
    cos_similarity = lambda a, b: np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    return cos_similarity(individual, target_word_bits)

### 3. Select Mating Pool

In [ ]:
def select_mating_pool(population, target_word_bits=target_word_bits, elite_size=50):
    '''
    Function:
    ---------
    Creates the 
    
    Parameters:
    -----------
    
    
    Returns:
    --------
    '''
    
    # Get sorted list of individuals in population
    # by cosine similarity with target_word_bits
    population_sorted_by_fitness = \
        sorted([(individual, fitness(individual)) for individual in population], 
               key=operator.itemgetter(1), 
               reverse=True)
    
    

### 4. Breed

### 5. Mutation 

---
## Covariance Matrix Adaptation - ES<a id='cma-es'></a>


---
## Particle Swarm Optimization 

---
### Resources:
- [Convert binary to ASCII and vice versa](https://stackoverflow.com/questions/7396849/convert-binary-to-ascii-and-vice-versa)
- [Evolution of a salesman: A complete genetic algorithm tutorial for Python](https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35)